# Introduction to Vcc Glitch Attacks

**This tutorial assumes the following hardware setup:**
* CW308_STM32F0 target
* Short SMA cable between CWLite and CW308
* 7.37MHz Xtal in X1 on CW308
* CW308 J3 set to Crystal

**This tutorial may be possible with other hardware setups, but workarounds may be needed**

## Background on Vcc (Power) Glitching

The previous clock glitching tutorials looked into the assumption of a constant clock. But instead we can modify the voltage of the device, causing for example a failure to correctly read a memory location or otherwise cause havoc with the proper functioning.

An example of a successful Vcc glitch is shown in the following figures (Vcc in blue, clock in red):

![](https://wiki.newae.com/images/4/4f/Vccglitch_working.png)

![](https://wiki.newae.com/images/6/60/Vccglitch_working_zoom.png)

Like with clock glitching, Vcc glitching is highly sensitive to glitch offset and width. While the above glitch was successful, the following was not:

![](https://wiki.newae.com/images/b/b6/Vccglitch_notworking_zoom.png)

Vcc glitching is also very sensitive to the shape of the glitch: things like board layout and the distance between where the glitch is inserted and the target can make the difference between successful and unsuccessful glitches.

Despite these additional complications, Vcc glitching is an extremely useful tool as it allows attacks on targets that do not run off of external clock inputs.

### Background on Glitch Generation

For more details, please see [Fault_1-Introduction_to_Clock_Glitching](Fault_1-Introduction_to_Clock_Glitch_Attacks.ipynb), this tutorials assumes you have already performed the clock glitching tutorial.

The glitch generation hardware is the same as used in the clock glitching attack. The generated glitches are synchronous to the device clock, and inserted at a precise offset from the clock edge.

Glitches can be inserted continuously or triggered by some event. The following figure shows the generation of two glitches:

![](https://wiki.newae.com/images/9/95/Glitchgen-mux-glitchonly.png)

The VCC glitching method here uses an electronic switch (a MOSFET) to short the power line to GND at specific instances. The following figure shows the basic function of this system:

![](https://wiki.newae.com/images/8/82/Glitch-vccglitcher.png)

This method allows use with the standard side-channel analysis development board, which has resistors inserted into the VCC lines already. The downside of this method is that it can only generate short glitches, since the power consumption through the shunt resistor will short out the resistor.

The MOSFET glitching hardware is built into the ChipWhisperer-Lite (both CW1173 and CW1180) board. The ChipWhisperer-Capture Rev2 uses an external VCC glitching board.

## Setting up Firmware

During this tutorial, we will once again be working off the `glitch-simple` project. We'll start by creating a new firmware project:

In [ ]:
%%bash
cd ../../hardware/victims/firmware/
mkdir -p glitch-simple-lab2 && cp -r glitch-simple/* $_

For this tutorial we'll be using the STM32F0 target, since that board often works well for Vcc glitching.

In [ ]:
PLATFORM = "CW308_STM32F0"
CRYPTO_TARGET="NONE"

Now navigate to the `glitch-simple-lab2` folder and open `glitchsimple.c` in a code editor. Find the `glitch_infinite()` function:

```C
void glitch_infinite(void)
{
    char str[64];
    unsigned int k = 0;
    //Declared volatile to avoid optimizing away loop.
    //This also adds lots of SRAM access
    volatile uint16_t i, j;
    volatile uint32_t cnt;
    while(1){
        cnt = 0;
        trigger_high();
        trigger_low();
        for(i=0; i<200; i++){
            for(j=0; j<200; j++){
                cnt++;
            }
        }
        sprintf(str, "%lu %d %d %d\n", cnt, i, j, k++);
        uart_puts(str);
    }
}
```

As you can see, this function enters into an infinite loop with two inner loops that increment three variables (`cnt`, `i`, and `j`).  These are sent back over serial along with an overall loop counter. During normal operation, we should receive `40000 200 200 $k` (where `$k` is the value of the loop counter `k`). Our objective will be to insert a Vcc glitch such that one or more of the numbers that we get back are incorrect.

Navigate to `main()` and change the `glitch1()` call to instead call `glitch_infinite()`. The below block will also do this:

In [ ]:
%%bash
# replaces glitch1() with glitch3()
cd ../../hardware/victims/firmware/glitch-simple-lab2
awk '{gsub(/glitch1\(\)/, "glitch_infinite\(\)")}1' glitchsimple.c > tmp.c 2>/dev/null
rm glitchsimple.c
mv tmp.c glitchsimple.c

Then build the firmware:

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/glitch-simple-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2

## Attack Script

### Setup

Now that we've studied the code and have an objective, we can start building our attack script. We'll start by connecting to and setting up the ChipWhisperer, then programming it. As usual, make sure you modify `fw_path` with the path to the file you built in the last step.

In [ ]:
%run "Helper_Scripts/CWLite_Connect.ipynb"

In [ ]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

In [ ]:
# uncomment based on your target
fw_path = "../../hardware/victims/firmware/glitch-simple-lab2/glitchsimple-CW308_STM32F0.hex"
%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/No_Programmer.ipynb"

In [ ]:
program_target(scope, fw_path)

Like with clock glitching, Vcc glitching may crash the target, requiring a reset:

In [ ]:
import time
def reset_target(scope):
    scope.io.nrst = 'low'
    #scope.io.pdic = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high'
    #scope.io.pdic = 'high'

Now that we have some of the basic setup done, let's make sure the firmware works as we expect. If we reset the target and wait a second, then print the serial data we got back, we should see a number of lines of the form `40000 200 200 $k`.

In [ ]:
reset_target(scope)
target.ser.flush()
time.sleep(1)
num_char = target.ser.inWaiting()
resp = target.ser.read(num_char)
print(resp)

### Glitch Setup

Since Vcc glitching requires greater precision than clock glitching, we'll need to take a different approach than we did with the last tutorial. 

Our first major change is to the clock of the ChipWhisperer. Instead of running at 7.37MHz, we'll instead run at a much higher clock rate (257.95MHz or 35\*7.37MHz). This will give us much better control over the width, shape, and placement of our glitches. Real targets often have their own indepentent clock setup anyway, so this isn't an obstacle to a real attack.

Note that this clock frequency is higher than the ADC on the ChipWhisperer-Lite can run off of, so you'll likely see the red ADC DCM Lock Failed LED on your ChipWhisperer light up.

In [ ]:
scope.clock.clkgen_freq = 257950000

Next, we'll setup the glitch module itself. Most of these settings should look familiar from the previous tutorial with a few new additions:

* Instead of setting the clock source for the target to be the glitch module, we instead set the low power MOSFET's input to be the glitch module by setting `scope.io.glitch_lp` to `True`. The ChipWhisperer-Lite also has a high power MOSFET, but we won't be using that in this tutorial.
* Instead of setting the glitch output to something like "clock_xor", we instead set it to "glitch_only", since we don't want Vcc of the target to be oscillating with our clock.

These settings work well with the CW308_STM32F0 target, but other targets will likely require different settings.

In [ ]:
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "glitch_only"
scope.glitch.trigger_src = "ext_single"
scope.glitch.width = 29
scope.glitch.offset = 10
scope.glitch.repeat = 14
scope.io.glitch_lp = True
print(scope.glitch)

### Glitching a Single Point

Unlike with the previous tutorial, we don't control when the device sends serial data back to us. This means we'll need to parse the data we get back. To make things a little simpler, we'll start off by attacking a single point multiple times.

In [ ]:
scope.glitch.ext_offset = 2180

We start our attack off by flushing the ChipWhipserer's serial buffer:

```Python
target.ser.flush()
```

Next, we'll set our trigger source to be "ext_continuous". This differs from "ext_single" in that the ChipWhisperer doesn't need to be armed to insert a glitch, making our loop a little simpler:

```Python
scope.glitch.trigger_src = "ext_continuous"
```

A key part of parsing the serial data is to be able to read a line of data (data terminated with "\n"). We can do that by reading back data until we get a newline character ("\n"):

```Python
while "\n" not in line:
    time.sleep(0.1)
    num_char = target.ser.inWaiting()
    line += target.ser.read(num_char)
```

This needs to be repeated twice in our loop: once at the start to make sure we're on a newline (so we don't look at the wrong numbers for our glitch) and again to actually read the line. For the first read, we also need to make sure we keep any characters after the newline, as this will be the start of the actual line we parse. All together, this looks like:

```Python
line = ""
while "\n" not in line:
    time.sleep(0.1)
    num_char = target.ser.inWaiting()
    line += target.ser.read(num_char)
lines = line.split("\n") 
if len(lines) > 1:
    line = lines[-1]
else:
    line = ""

while "\n" not in line:
    time.sleep(0.1)
    num_char = target.ser.inWaiting()
    line += target.ser.read(num_char)
```

Now that we have our line of data we can parse it by splitting it up via spaces to get each number. 

After the loop ends, we'll need to set our trigger back to "ext_single" to stop the glitches from continuing.

All together (with some additional error checking), this looks like:

In [ ]:
from tqdm import tnrange
reset_target(scope)
target.ser.flush()
#scope.glitch.trigger_src = "ext_single"
scope.glitch.trigger_src = "ext_continuous"

for j in tnrange(100):
    line = ""
    while "\n" not in line:
        time.sleep(0.1)
        num_char = target.ser.inWaiting()
        line += target.ser.read(num_char)
    lines = line.split("\n") 
    if len(lines) > 1:
        line = lines[-1]
    else:
        line = ""
    
    while "\n" not in line:
        time.sleep(0.1)
        num_char = target.ser.inWaiting()
        line += target.ser.read(num_char)
    
    nums = line.split(" ")
    if nums[0] == "":
        continue
    if int(nums[0]) != 40000:
        print(line)
    
scope.glitch.trigger_src = "ext_single"

You should see one or two lines printed above. If not, don't worry: we'll be changing the loop to scan offsets, making the glitch more reliable.

Adding the ability to scan ext_offset is pretty simple. We really just need to put the code we had before into another loop:

In [ ]:
from tqdm import tnrange
target.ser.flush()
for ext in tnrange(2316, 2319, desc="Ext offset"):
    scope.glitch.ext_offset = ext
    target.ser.flush()
    scope.glitch.trigger_src = "ext_continuous"

    for j in tnrange(100, leave=False, desc="Glitch attempt"):
        line = ""
        while "\n" not in line:
            time.sleep(0.1)
            num_char = target.ser.inWaiting()
            if num_char == 0:
                reset_target(scope)
                target.ser.flush()
                continue
            line += target.ser.read(num_char)
        lines = line.split("\n") 
        if len(lines) > 1:
            line = lines[-1]
        else:
            line = ""

        while "\n" not in line:
            time.sleep(0.1)
            num_char = target.ser.inWaiting()
            if num_char == 0:
                reset_target(scope)
                target.ser.flush()
                continue
            line += target.ser.read(num_char)

        nums = line.split(" ")
        if int(nums[0]) != 40000:
            print("Glitched @ {}".format(ext))
            print("Text: " + line)
        elif len(nums) > 1 and int(nums[1]) != 200:
            print("Glitched @ {}".format(ext))
            print("Text: " + line)
        elif len(nums) > 2 and int(nums[2]) != 200:
            print("Glitched @ {}".format(ext))
            print("Text: " + line)
        #t.update()
        #print(line, end = "")

    scope.glitch.trigger_src = "ext_single"
    
#t.close()

There's a few good ranges you can try here that may produce different results:
* [2036, 2038]
* [2180, 2182]

These ranges all only affect the first number, so you may also want to try a large ext offset range to try to get even more glitch effects.

The tutorial is now done, so we'll disconnect from the ChipWhisperer:

In [ ]:
scope.dis()
target.dis()

## Conclusion

With the tutorial now finished, you should have some Vcc glitching experience under your belt. If you're interested in doing more Vcc glitching, you may want to try [Tutorial A9](https://wiki.newae.com/Tutorial_A9_Bypassing_LPC1114_Read_Protect) from the ChipWhisperer Wiki, which uses Vcc glitching to bypass code readout protection on an LPC1114 (requires an LPC1114 dev board). You may also want to try glitching some of the other functions in `glitchsimple.c`. If you have a Raspberry Pi, you can also attempt the attack described [here](https://wiki.newae.com/Tutorial_A3_VCC_Glitch_Attacks#Glitching_More_Advanced_Targets:_Raspberry_Pi) (though you'll need to transfer the steps from the old GUI over to Jupyter).